# Create a neural network to solve the MNIST problem using Keras using ChatGPT

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# normalize the pixel values (0-255) to be between 0 and 1
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

# define the model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
          activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

# compile the model
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

X_val = X_test[:2000]
y_val = y_test[:2000]
X_test = X_test[2000:]
y_test = y_test[2000:]

# train the model
history = model.fit(X_train, y_train, batch_size=32,
                    epochs=10, validation_data=(X_val, y_val))


2023-02-17 09:20:13.488303: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
1875/1875 [==============================] - 99s 52ms/step - loss: 0.1448 - accuracy: 0.9554 - val_loss: 0.0758 - val_accuracy: 0.9770
Epoch 2/10
1875/1875 [==============================] - 96s 51ms/step - loss: 0.0491 - accuracy: 0.9854 - val_loss: 0.0624 - val_accuracy: 0.9780
Epoch 3/10
1875/1875 [==============================] - 103s 55ms/step - loss: 0.0312 - accuracy: 0.9904 - val_loss: 0.0495 - val_accuracy: 0.9840
Epoch 4/10
1875/1875 [==============================] - 105s 56ms/step - loss: 0.0232 - accuracy: 0.9929 - val_loss: 0.0505 - val_accuracy: 0.9835
Epoch 5/10
1875/1875 [==============================] - 104s 55ms/step - loss: 0.0158 - accuracy: 0.9949 - val_loss: 0.0517 - val_accuracy: 0.9840
Epoch 6/10
1875/1875 [==============================] - 104s 55ms/step - loss: 0.0115 - accuracy: 0.9965 - val_loss: 0.0506 - val_accuracy: 0.9845
Epoch 7/10
1875/1875 [==============================] - 103s 55ms/step - loss: 0.0092 - accuracy: 0.9972 - val_loss: 0.0

In [2]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


250/250 [==============================] - 4s 15ms/step - loss: 0.0396 - accuracy: 0.9912
Test accuracy: 0.9912499785423279
